## Title :

Case Study: Covid-19

## Hints: 

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html" target="_blank">sklearn.model_selection.GridsearchCV</a>
Exhaustive search over specified parameter values for an estimator.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html" target="_blank">sklearn.metrics.confusion_matrix</a>
Compute confusion matrix to evaluate the accuracy of a classification.

<a href="https://seaborn.pydata.org/generated/seaborn.heatmap.html" target="_blank">seaborn.heatmap</a>
Plot rectangular data as a color-encoded matrix.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html?highlight=roc_auc_score#sklearn.metrics.roc_auc_score" target="_blank">sklearn.metrics.roc_auc_score</a>
Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html?highlight=roc%20curve#sklearn.metrics.roc_curve" target="_blank">sklearn.metrics.roc_curve</a>
Compute Receiver operating characteristic (ROC).

# COVID-19 Machine Learning Dataset®

>Adopted from the dataset provided by Dr. Karandeep Singh [@kdpsinghlab](https://twitter.com/kdpsinghlab/status/1239416911668092928)

The goal of this case study (intended for education) is to **predict the urgency** with which a COVID-19 patient will need to be admitted to the hospital from the time of onset of symptoms.

The original dataset is located on this [github repo](https://github.com/ml4lhs/covid19_ml_education/raw/master/covid_ml.csv). Please note that this dataset has been simplified for this case study

The raw data comes from the following [source](http://virological.org/t/epidemiological-data-from-the-ncov-2019-outbreak-early-descriptions-from-publicly-available-data/337).

## Intended For Educational Use Only
## Should this data be used for research?

No. Students working with this dataset should understand that both the source data and the ML data have several limitations:
- The source data is crowdsourced and may contain inaccuracies.
- There may be duplicate patients in this dataset
- There is a substantial amount of missingness in the symptoms data.

## **And most importantly:**
- The entire premise is flawed. The fact that a patient was admitted the same day as experiencing symptoms may have more to do with the availability of hospital beds as opposed to the patient's acuity of illness.
- Also, the fact that less sick patients or asymptomatic patients may not have been captured in the source dataset mean that the probabilities estimated by any model fit on this data are unlikely to reflect reality.

**Primary predictors:**
- age (if an age range was provided in the source data, only the first number is used)
- sex
- cough, fever, chills, sore_throat, headache, fatigue (all derived from the symptoms column using regular expressions)

>The goal of the exercise is to make a classification model to predict the **urgency_of_admission** based on the following criteria
1. 0-1 days from onset of symptoms to admission => High
2. 2+ days from onset of symptoms to admission *or* no admission => Low
  

In [58]:
# Import necessary libraries
# Feel free to import other modules and libraries as you deem fit
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import classification_report,roc_auc_score, roc_curve, accuracy_score

%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"


## Calling the dataset
>We are using a modified dataset found here [covid_19 dataset source](https://web.us.edusercontent.com/hulb749ekf8a0pncvj99orb78k)

The following changes were made:
1. Categorical values changed to 1 and 0
2. SMOTE used to upsample in order to balance the dataset. Refer to the class imbalance exercise in the Random Forest session.

In [59]:
# Read the train and test data
# Take a look at the data to understand the features and reponse

# Your code here
df = pd.read_csv('covid_data.csv')
df.head(5), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   age          1252 non-null   int64
 1   sex          1252 non-null   int64
 2   cough        1252 non-null   int64
 3   fever        1252 non-null   int64
 4   chills       1252 non-null   int64
 5   sore_throat  1252 non-null   int64
 6   headache     1252 non-null   int64
 7   fatigue      1252 non-null   int64
 8   Urgency      1252 non-null   int64
dtypes: int64(9)
memory usage: 88.2 KB


(   age  sex  cough  fever  chills  sore_throat  headache  fatigue  Urgency
 0   30    1      0      0       0            0         0        0        0
 1   47    1      0      0       0            0         0        0        0
 2   49    1      0      0       0            0         0        0        0
 3   50    0      0      0       0            0         0        0        0
 4   59    0      0      1       0            0         0        0        0,
 None)

In [60]:
# define X_train, y_train, X_test, and y_test 
# Urgency is the response variable, all other variables are the predictors                        

# Your code here
X = df[['age', 'sex', 'cough', 'fever', 'chills', 'sore_throat', 'headache', 'fatigue']]
y = df['Urgency']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## GridsearchCV for Logistic Regression
> Perform a hyper-parameter search to get the best C value for Logistic Regression using [GridsearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
>> For simplicity, use **accuracy** as the metric to choose best hyper-parameter


In [61]:
# Perform GridSearchCV to get the best C value for a Logistic Regression model
# Feel free to use the cv and set of C values of your choice
# Remember to keep track of your best C value

# Your code here
parameters = [{'C':list(x/10.0 for x in range(0, 10))}]
grid_search = GridSearchCV(estimator = LogisticRegression(),  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)
grid_search.fit(X_train, y_train) 

best_model = grid_search.best_estimator_
best_model

C:\Users\TLG User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\TLG User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.or

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                0.9]}],
             scoring='accuracy')

LogisticRegression(C=0.8)

### Fitting the data and making predictions


In [67]:
# Using the C value from above, initialize a Logistic Regression model
# Fit the model on the train data
# Predict on the test data

# Your code here
logreg = LogisticRegression(C=0.8)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

C:\Users\TLG User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.8)

In [68]:
# Compute the accuracy of the model
logistic_acc = accuracy_score(y_test, y_pred)
logistic_acc


0.6294820717131474

## GridsearchCV for KNN classification
> Perform a hyper-parameter search to get the best k value for KNN Classification using [GridsearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
>> For simplicity, use **accuracy** as the metric to choose best hyper-parameter


In [71]:
# Perform GridSearchCV to get the best k value for a kNN Classification model
# Feel free to use the cv and set of k values of your choice
# Remember to keep track of your best k value

# Your code here
parameters = [{'n_neighbors':list(range(1, 20))}]
grid_search = GridSearchCV(estimator = KNeighborsClassifier(),  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)
grid_search.fit(X_train, y_train) 

pd.DataFrame(grid_search.cv_results_)[['mean_test_score', 'std_test_score', 'params']], print(grid_search.best_params_)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19]}],
             scoring='accuracy')

{'n_neighbors': 3}


(    mean_test_score  std_test_score               params
 0          0.622378        0.036401   {'n_neighbors': 1}
 1          0.598378        0.036376   {'n_neighbors': 2}
 2          0.630373        0.021138   {'n_neighbors': 3}
 3          0.609398        0.025445   {'n_neighbors': 4}
 4          0.626403        0.032552   {'n_neighbors': 5}
 5          0.617388        0.027770   {'n_neighbors': 6}
 6          0.627393        0.028095   {'n_neighbors': 7}
 7          0.620378        0.024710   {'n_neighbors': 8}
 8          0.607398        0.023045   {'n_neighbors': 9}
 9          0.629423        0.038588  {'n_neighbors': 10}
 10         0.621413        0.025787  {'n_neighbors': 11}
 11         0.627418        0.033577  {'n_neighbors': 12}
 12         0.616408        0.016836  {'n_neighbors': 13}
 13         0.622433        0.031423  {'n_neighbors': 14}
 14         0.615458        0.043037  {'n_neighbors': 15}
 15         0.625418        0.023304  {'n_neighbors': 16}
 16         0.

### Fitting the data and making predictions


In [72]:
# Using the k value from above, initialize a kNN Classification model
# Fit the model on the train data
# Predict on the test data

# Your code here
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)

KNeighborsClassifier(n_neighbors=3)

In [73]:
# Compute the accuracy of the model
knn_acc = accuracy_score(y_test, y_pred_knn)
knn_acc

0.5737051792828686

In [76]:
# Store the Confusion Matrix of the trained Logistic Regression Model on the test data in a variable


# Your code here
cm_logreg = confusion_matrix(y_test, y_pred)
cm_logreg

array([[68, 56],
       [37, 90]], dtype=int64)

In [77]:
# Store the Confusion Matrix of the trained kNN Classification Model on the test data in a variable

# Your code here
cm_knn = confusion_matrix(y_test, y_pred_knn)
cm_knn

array([[65, 59],
       [48, 79]], dtype=int64)

## What is a Confusion Matrix?

 A classifier will get some samples right, and some wrong. Generally we see which ones it gets right and which ones it gets wrong on the test set
 
 ![hwimages](images/confusionmatrix.png)
 

### True Positive
- Samples that are +ive and the classifier predicts as +ive are called True Positives (TP)

### False Positive
- Samples that are -ive and the classifier predicts (wrongly) as +ive are called False Positives (FP)

### True Negative
- Samples that are -ive and the classifier predicts as -ive are called True Negatives (TN)

### False Negative
- Samples that are +ive and the classifier predicts as -ive are called False Negatives (FN)

### <ins>The boy who cried wolf: Data Science edition</ins>

#### Predicted wolf, but no wolf 

 ![hwimages](images/fp.jpeg)

#### Predicted no wolf, but actually wolf

 ![hwimages](images/fn.jpeg)

### Plot the Confusion Matrix

Plot the Confusion Matrix for each of the above models as a heatmap. Your plot should look similar to the following:

 ![hwimages](images/cm_plot.png)


In [0]:
# Plot of the Confusion Matrix for the Logisitic Regression and kNN Classification model

# Your code here


## **Sensitivity**

The **Sensitivity**, also known as **Recall** or **True Positive Rate(TPR)**


$$TPR = Recall = \frac{TP}{OP} = \frac{TP}{TP+FN},$$

also called the **Hit Rate**: the fraction of observed positives (1s) the classifier gets right, or how many true positives were recalled. Maximizing the recall towards 1 means keeping down the false negative rate

In [0]:
# Compute the Sensitivity for the Logistic Regression model
logistic_recall = ___

# Compute the Sensitivity for the kNN Classification model
knn_recall = ___


## **Specificity**
The **Specificity** or **True Negative Rate** is defined as

$$TNR = \frac{TN}{ON} = \frac{TN}{FP+TN}$$

In [0]:
# Compute the Specificity for the Logistic Regression model
logistic_fpr = ___

# Compute the Specificity for the kNN Classification model
knn_fpr = ___


## **Precision** (Positive Predicted Value)

**Precision**,tells you how many of the predicted positive(1) hits were truly positive

$$Precision = \frac{TP}{PP} = \frac{TP}{TP+FP}.$$

In [0]:
# Compute the Precision for the Logistic Regression model
logistic_precision = ___

# Compute the Precision for the kNN Classification model
knn_precision = ___


## **F1 score**
**F1** score gives us the Harmonic Mean of Precision and Recall.
It tries to minimize both **false positives** and **false negatives** simultaneously

$$F1 =  \frac{2*Recall*Precision}{Recall + Precision}$$

In [0]:
# Compute the F1-Score for the Logistic Regression model
logistic_fscore = ___

# Compute the F1-Score for the kNN Classification model
knn_fscore = ___


In [0]:
# Helper code to bring everything together
pt = PrettyTable()

pt.field_names = ["Metric", "Logistic Regression", "kNN Classification"]
pt.add_row(["Accuracy", round(logistic_acc, 3), round(knn_acc, 3)])
pt.add_row(["Sensitivity(Recall)", round(logistic_recall, 3), round(knn_recall, 3)])
pt.add_row(["Specificity", round(logistic_fpr, 3), round(knn_fpr, 3)])
pt.add_row(["Precision", round(logistic_precision, 3), round(knn_precision, 3)])
pt.add_row(["F1 Score", round(logistic_fscore, 3), round(knn_fscore, 3)])

print(pt)


# BACK TO THE LECTURE

## Bayes Theorem & Diagnostic testing

Refer to Dr. Rahul Dave's [Covid19 Serological testing blog](https://covid19.posts.ai/2020/04/04/bayes-rule-and-serological-testing.html) for an excellent introduction to all the above concepts.

In [0]:
# Compute the area under the ROC curve for the Logistic Regression model
logreg_auc = ___

# Compute the area under the ROC curve for the kNN Classification model
knnreg_auc = ___


## ROC Curve

To make a ROC curve you plot the True Positive Rate, against the False Positive Rate,

The curve is actually a 3 dimensional plot, which each point representing a different value of threshold.

![ROC](images/roc.png)


In [0]:
# Plot the ROC curve for the Logistic Regression model and kNN Classification model
# You can refer to the end of homework 5 for example code

# Your code here


# Which classifier to choose?

## Choice of classifier Scenario 1 - BRAZIL

<img src="images/brazil.png" alt="drawing" width="500"/>

>**The new variant of the Covid-19 virus is contagious and is infecting many Brazilians**

>>*Brazilian officials however dictate that hospitals do not classify a large number of people at 'high' risk to avoid bad press and subsequent political global backlash**

**In numbers we need the best classifier with the following restriction**

$$TPR + FPR \le 0.5$$     


## Choice of classifier Scenario 2 - GERMANY

<img src="images/germany.png" alt="drawing" width="500"/>

>**It's the month of February, and Germany, is now aware that the pandemic of Covid-19 has severely hit Europe. Italy is already decimated and there is suspected spread to other European nations as well**

>> German officials have a clear target. The want the fatality ratio to be as less as possible. Thus, it is imperative to find cases in need of urgent attention and give them the best chance of survival.

**In numbers we need the best classifier with the following restriction**

$$ 0.8 \ge TPR \le 0.85 $$

# Choice of classifier Scenario 3 - INDIA

<img src="images/india.png" alt="drawing" width="500"/>

>**It's the month of May, and India, now severly impacted by Covid-19, has now run a major shortage of hospital beds for suspected cases**

>> Owing to exponentially rising cases Indian officials cannot afford many **False Positives** to be given hospital beds. Hence, it is dictated that hospitals do not classify a large number of people at 'high' risk to avoid bed shortage for **At risk** patients

>> India has only 1 million beds left, and there are already 2 million people suspected of having the disease. The officials need to work out a strategy to find the people at most need of urgent care.

**In numbers we need the best classifier with the following restriction**

$$TP + FP = 1000000$$

$$TP = TPR*OP$$ $$FP = TPR*ON$$

$$TPR*OP + FPR*ON = 1000000$$

$$Assuming\ OP=ON = 500000$$

$$TPR + FPR \le 1 $$

## ROC curve with boundary conditions

Plot the ROC curve of the Logistic Regression model and kNN Classification model, along with the boundary conditions for each of the scenarios.

Each of the scenarios can be represented as a region governed by straight line(s) based on the given conditions. The resulting plot will look similar to the following:

<img src="images/final.png" alt="drawing" width="900"/>


In [0]:
# Area under curve - Logistic regression & kNN
# along with the boundary conditions

# Your code here


# BACK TO THE LECTURE